# Репрезентация текста в ML-моделях

![](https://pbs.twimg.com/media/EZlnU2oXsAEMUSP?format=jpg&name=4096x4096)

В первом модуле мы научились предобрабатывать тексты: лемматизировать, выделять морфологическую информацию, создавать семантические векторы с помощью w2v-моделей, записывать полученную информацию в датафрейм. Тем не менее, модели машинного обучения плохо понимают тексты — гораздо лучше они справляется с цифрами. 

Способов **векторизации** текста (то есть перехода от _слов_ к _значащим числам_ ) есть несколько. Сегодня мы рассмотрим TF-IDF.

## Как вообще можно представлять текст?

Основных способов на самом деле два: мешок слов / bag of words и использование эмбеддингов.

---

**Мешок слов** — способ, основанный на подсчёте частотностей слов в конкретном корпусе.

<font style="color:green">Плюсы:</font> учитывает нюансы вашей коллекции текстов; проще для понимания.

<font style="color:red">Минусы:</font> нужно готовить самому; если в вашей коллекции появится новый документ с новыми словами, с ним будет трудно — придётся всё пересчитывать.

---

**Эмбеддинги** — заранее предобученные готовые модели, которые могут конвертировать ваш текст в вектор.

<font style="color:green">Плюсы:</font> быстро; универсально.

<font style="color:red">Минусы:</font> модели занимают много места; иногда от модели зависит итоговый вектор и она сильно влияет на качество.

## TF-IDF

Одна из самых популярных метрик — TF-IDF. Она не только учитывает все слова в тексте, но и принимает во внимание их важность.

### TF

В этой аббревиатуре TF обозначает **text frequency** — сколько раз слово (а вернее, «терм») встретилось в документе. Считается просто — количество раз, которое слово встретилось, делённое на общее количество слов в тексте:

$$TF_{i,j} = \frac{n_{i,j}}{\sum_{k}^{}n_{i,j}}$$

### IDF

IDF — это **inverse document frequency** — важность терма для конкретного документа. Он считается так:

$$ IDF = log \frac{N}{n_t}$$

— общее количество документов делится на количество документов, где нужный терм встретился хотя бы один раз; от полученной величины берётся логарифм. Таким образом, если терм встретился только в одном документе, то его IDF будет высокой (=> это важный для документа терм).

### Общая мера

Итоговая величина получается перемножением исходных двух метрик:

$$TFIDF = TF \cdot IDF$$

### Как это работает?

Возьмём два текста:

* `Мама мыла раму`

* `Даша мыла яблоки`

— всего в них 5 уникальных слов. 

Получается матрица 5 (слов) x 2 (документа):

||мама|мыла|раму|даша|яблоки|
|:--:|:--:|:--:|:--:|:--:|:--:|
|документ 1||||||
|документ 2||||||

Посчитаем TF:

||мама|мыла|раму|даша|яблоки|
|:--:|:--:|:--:|:--:|:--:|:--:|
|документ 1|1/3|1/3|1/3|0|0|
|документ 2|0|1/3|0|1/3|1/3|

Посчитаем IDF:

||мама|мыла|раму|даша|яблоки|
|:--:|:--:|:--:|:--:|:--:|:--:|
|документ 1|$log(2/1)$|$log(2/2)$|$log(2/1)$|||
|документ 2||$log(2/2)$||$log(2/1)$|$log(2/1)$|

Перемножаем:

||мама|мыла|раму|даша|яблоки|
|:--:|:--:|:--:|:--:|:--:|:--:|
|документ 1|0.231|0|0.231|0|0|
|документ 2|0|0|0|0.231|0.231|

Соответственно, документ 1 описывается вектором `[0.231, 0, 0.231, 0, 0]`, а документ 2 — `[0, 0, 0, 0.231, 0.231]`.

### Что важно помнить при подсчёте TF-IDF?

**Преобработанный текст**

Ваш текст уже должен быть избавлен от знаков препинания, неуместных заглавных букв и т.д., а также лемматизирован. Так вы избавитесь от разных «термов»-форм слова, которые значат по сути одно и то же.

**Стоп-слова**

При работе с TF-IDF матрицей лучше убирать из текста стоп-слова: их высокая частотность и присутствие во многих документах может размыть вам картинку.

### Реализация TF-IDF

…уже сделана за нас в пакете sklearn.

In [1]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
stops = stopwords.words("russian")

tfidf = TfidfVectorizer(
    analyzer="word", # анализировать по словам или по символам (char)
    stop_words=stops # передаём список стоп-слов для русского из NLTK
)

## Практика

Загрузим данные из некоторой заполярной газеты:

In [3]:
import os

In [4]:
path_to_articles = "./data/polar_circle"
articles = [os.path.join(path_to_articles, item) 
            for item in os.listdir(path_to_articles)
            if item.endswith(".txt")]

In [5]:
articles_texts = []

for article_path in articles:
    with open(article_path, "r", encoding="utf-8") as a_src:
        # первые 6 строк — метаданные, последняя — число комментариев
        articles_texts.append("\n".join(a_src.readlines()[6:-1]))

Лемматизируем:

In [6]:
from nltk.tokenize import wordpunct_tokenize
from pymorphy2 import MorphAnalyzer

In [7]:
morph = MorphAnalyzer()

In [8]:
articles_preprocessed = []
for a_text in articles_texts:
    a_tokens = wordpunct_tokenize(a_text)
    a_lemmatized = " ".join([morph.parse(item)[0].normal_form for item in a_tokens])
    articles_preprocessed.append(a_lemmatized)

Пропускаем через TF-IDF:

In [9]:
articles_tfidf = tfidf.fit_transform(articles_preprocessed)
print(f"Матрица на {articles_tfidf.shape[0]} документов и {articles_tfidf.shape[1]} термов")

Матрица на 25 документов и 2283 термов


При помощи метода `tfidf.get_feature_names()` можно посмотреть, какие именно термы есть в вашей матрице (здесь этого не будет, т.к. термов очень много).

In [10]:
# проверим, правда ли 2000+ термов
len(tfidf.get_feature_names())

2283

## А делать-то с этим что?

Вариант 1: **машинное обучение**. Полученную матрицу можно засунуть в ML-модель для предсказания чего бы то ни было.

![](https://i.pinimg.com/originals/1e/11/31/1e1131d4cac5a6afb49580dab10aa281.jpg)

Вариант 2: **keyword extraction на коленке**. 

Задача _извлечения ключевых слов_ (keyword extraction) заключается в том, чтобы описать текст некоторым количеством слов, которые максимально точно передают его содержание. Попробуем написать функцию, которая для каждого ряда будет брать N слов с наибольшим TF-IDF и выводить их.

In [11]:
import numpy as np

In [12]:
def get_top_tf_idf_words(tfidf_vector, feature_names, top_n):
    sorted_nzs = np.argsort(tfidf_vector.data)[:-(top_n+1):-1]
    return feature_names[tfidf_vector.indices[sorted_nzs]]

In [13]:
feature_names = np.array(tfidf.get_feature_names())

for i, article in enumerate(articles_texts):
    # напечатаю только первые 5 статей
    if i < 5:
        article_vector = articles_tfidf[i, :]
        words = get_top_tf_idf_words(article_vector, feature_names, 10)
        print(article)
        print(words)

В городской администрации прошло очередное заседание антинаркотической комиссии.

Важнейшей темой обсуждения стало состояние наркоситуации в городе, работа по предупреждению, выявлению и раскрытию преступлений, связанных с незаконным оборотом наркотиков на территории города.

По аналитическим данным, озвученным временно исполняющим обязанности начальника полиции ОМВД России по городу Салехарду Андреем Печёнкиным, за первое полугодие этого года на профилактический учёт поставлено 27 человек. В прошлом году за такой же период – на десять больше. По мнению специалистов, снижению способствует и активизация борьбы с распространением наркотиков, и расширение судебной практики в отношении лиц, занимающихся распространением. А также профилактические мероприятия. В то же время происходит рост числа граждан, употребляющих наркотические вещества (токсикомания). За первое полугодие под наблюдение взято 11 человек, большинство из них подростки.

Правоохранительными органами с начала года в городе в